In [2]:
#---------------import thư viên--------------------
import numpy as np
import pandas as pd

from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Bidirectional,  Dropout, Reshape
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.metrics import mean_squared_error
from keras.regularizers import l2
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from IPython.display import display
from IPython.display import SVG, Image
from keras.utils.vis_utils import model_to_dot
#---------------------------------


Using TensorFlow backend.


In [6]:
BATCH_SIZE = 38
LAG_SIZE = 1
TRAINING_SIZE = 400

raw_data = pd.read_csv('GSPC.csv')
raw_data['Date'] = pd.to_datetime(raw_data['Date'])
# raw_data = raw_data[::-1]
def preprocess(data):
    X_COLUMNS = ['Open', 'High', 'Low', 'Close','Volume']
    Y_COLUMNS = ['y']
    COLUMNS = X_COLUMNS + Y_COLUMNS
    
    
    scaler = StandardScaler()
    data[X_COLUMNS] = scaler.fit_transform(data[X_COLUMNS])
    
    
    data['y'] = data.Close.shift(-1)
    data = data.dropna()
    
    # BATCH
    data = data[:data.shape[0]-data.shape[0]%BATCH_SIZE]
    data = data[COLUMNS].as_matrix()

    # Split X and Y
    N = data.shape[0]-BATCH_SIZE -1
    data_x = np.zeros((N, BATCH_SIZE, 5))
    data_y = np.zeros((N, 1))
    print('N:' , N)
    
#     for i in range(N):
#         end = i + BATCH_SIZE + 1
#         series = data[i:end]
        
#         data_x[i, :] = series[:BATCH_SIZE, :-1].reshape(BATCH_SIZE, 2)
#         data_y[i] = series[BATCH_SIZE:, -1]
    
    return scaler, data_x, data_y

def split_train_test(data_x, data_y, test_size=0.2):
    train_size = int(data_y.shape[0] * (1-test_size))
    print('train_size:', train_size)
    
    train_x, test_x = data_x[:train_size], data_x[train_size:]
    train_y, test_y = data_y[:train_size], data_y[train_size:]
        
    return train_x, train_y, test_x, test_y
    
scaler, data_x, data_y = preprocess(raw_data)
train_x, train_y, test_x, test_y = split_train_test(data_x, data_y)

print('train_x:', train_x.shape)
print('train_y:', train_y.shape)
print('test_x :', test_x.shape)
print('test_y :', test_y.shape)

N: 16947
train_size: 13557
train_x: (13557, 38, 5)
train_y: (13557, 1)
test_x : (3390, 38, 5)
test_y : (3390, 1)


D:\Anaconda3\envs\tensorflow1\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [7]:
models = [GaussianNB(),
          SVC(random_state=5),
          RandomForestClassifier(random_state=5),
          MLPClassifier(random_state=5)]

for model in models:
    model.fit(train_x, train_y)
    predictions = model.predict(test_x)

ValueError: Found array with dim 3. Estimator expected <= 2.